# 定义新的数据类型
- 使用`data`关键字可以定义新的数据类型:

In [1]:
data BookInfo = Book Int String [String]
                deriving (Show)

- BookInfo 就是新类型的名字，我们称为类型构造器。类型构造器用于指代(refer)类型。
- Book则是值构造器（也称数据构造器）的名字。
- 类型构造器和值构造器的首字母都需要大写
- 在Book之后的Int,String和[String]是类型的组成部分。组成部分的作用，和面向对象语言中类的域作用一致：它是一个存储值的槽。（通常也将组成部分称为域。）
- 类型包含的成分结构相同但是不可用混用，比如我们定义一个`MagzineInfo`类型的成分和BookInfo一模一样，但是Haskell会将它们作为不同的类型来区别对待，因为他们的类型构造器和值构造器并不相同:
- 在ghci中我们可以使用:info查看更多关于给定表达式的信息，使用:type命令，可以查看值构造器的类型签名，了解他们是如果创造出类型的。

## 类型构造器和值构造器的命名

- 在Haskell里，类型的名字(类型构造器)和值构造器的名字是相互独立的。
- 类型构造器只能出现在类型的定义，或者类型签名当中。而值构造器只能出现在具体的代码中。
- 所以给类型构造器和值构造器赋予相同名字是没问题的，但是应当注意两者是不同的概念。而大部分oop语言是混杂到一起的。

## 类型别名

- 可以使用类型别名，来为一个已存在的类型设置一个更具描述性的名字
- 比如:

In [2]:
type CustomerID =  Int
type ReviewBody = String

data BookReview = BookReview BookInfo CustomerID ReviewBody

- 与oop里面使用变量名来赋予某个字段具体的含义相比，这样无疑是更加优越的，比如人名和书名都是String,可是确实应当有能力使两者不混为一谈
- 还可以为复杂的类型设置一个简短的名字:

In [3]:
type BookRecord = (BookInfo,BookReview)

- 类型别名只是为已有类型提供一个新名字而已，创建值的工作仍然由原来的类型的值构造器进行。(可以类比C/C++中的typedef)

# 代数数据类型
- Bool类型是代数数据类型(algebraic data type)的最简单，最常见的例子，一个代数数据类型可以有多于一个值构造器:

In [4]:
data Bool = False | True

上面代码定义的 Bool 类型拥有两个值构造器，一个是 True ，另一个是 False 。每个值构造器使用 | 符号分割，读作“或者” —— 以 Bool 类型为例子，我们可以说， Bool 类型由 True 值或者 False 值构成。

当一个类型拥有一个以上的值构造器时，这些值构造器通常被称为“备选”（alternatives）或“分支”（case）。同一类型的所有备选，创建出的的值的类型都是相同的。

代数数据类型的各个值构造器都可以接受任意个数的参数。[译注：不同备选之间接受的参数个数不必相同，参数的类型也可以不一样。]以下是一个账单数据的例子：

In [5]:
type CardHolder = String
type CardNumber = String
type Address = [String]
data BillingInfo = CreditCard CardNumber CardHolder Address
                 | CashOnDelivery
                 | Invoice CustomerID
                   deriving (Show)

这个程序提供了三种付款的方式。如果使用信用卡付款，就要使用 CreditCard 作为值构造器，并输入信用卡卡号、信用卡持有人和地址作为参数。如果即时支付现金，就不用接受任何参数。最后，可以通过货到付款的方式来收款，在这种情况下，只需要填写客户的 ID 就可以了。

当使用值构造器来创建 BillingInfo 类型的值时，必须提供这个值构造器所需的参数：

```
Prelude> :load BookStore.hs
[1 of 1] Compiling Main             ( BookStore.hs, interpreted )
Ok, modules loaded: Main.

*Main> :type CreditCard
CreditCard :: CardNumber -> CardHolder -> Address -> BillingInfo

*Main> CreditCard "2901650221064486" "Thomas Gradgrind" ["Dickens", "England"]
CreditCard "2901650221064486" "Thomas Gradgrind" ["Dickens","England"]

*Main> :type it
it :: BillingInfo
```
如果输入参数的类型不对或者数量不对，那么引发一个错误：

```
*Main> Invoice

<interactive>:7:1:
    No instance for (Show (CustomerID -> BillingInfo))
        arising from a use of `print'
    Possible fix:
        add an instance declaration for (Show (CustomerID -> BillingInfo))
    In a stmt of an interactive GHCi command: print it

```
ghci 抱怨我们没有给 Invoice 值构造器足够的参数。

## 元组和代数数据类型的使用场景
元组和自定义代数数据类型有一些相似的地方。比如说，可以使用一个 (Int, String, [String]) 类型的元组来代替 BookInfo 类型：
```
*Main> Book 2 "The Wealth of Networks" ["Yochai Benkler"]
Book 2 "The Wealth of Networks" ["Yochai Benkler"]

*Main> (2, "The Wealth of Networks", ["Yochai Benkler"])
(2,"The Wealth of Networks",["Yochai Benkler"])
```

代数数据类型使得我们可以在结构相同但类型不同的数据之间进行区分。然而对于元组来说，要元素的结构和类型都一致，那么元组的类型就是相同的,而对于两个不同的代数数据类型来说，即时值构造器成分的结构和类型都相同，它们也是不同的类型。

## 其他语言里类似代数数据类型的东西
- 代数数据类型为描述数据类型提供了一种单一且强大的方式。很多其他语言，要达到相当于代数数据类型的表达能力，需要同时使用多种特性。
- 以下用C/C++举例

### 结构
- 当只有一个值构造器时，代数数据类型和元组很相似：它将一系列相关的值打包成一个复合值。这种做法相当于 C 和 C++ 里的 struct ，而代数数据类型的成分则相当于 struct 里的域。

- 以下是一个 C 结构，它等同于我们前面定义的 BookInfo 类型：
```
struct book_info {
    int id;
    char *name;
    char **authors;
};
```
- 目前来说， C 结构和 Haskell 的代数数据类型最大的差别是，代数数据类型的成分是匿名且按位置排序的：

```
data BookInfo = Book Int String [String]
                deriving (Show)
```
- 按位置排序指的是，对成分的访问是通过位置来实行的，而不是像 C 那样，通过名字：比如 book_info->id 。

- 稍后的“模式匹配”小节会介绍如何访代数数据类型里的成分。在“记录”一节会介绍定义数据的新语法，通过这种语法，可以像 C 结构那样，使用名字来访问相应的成分。

### 枚举
C 和 C++ 里的 enum 通常用于表示一系列符号值排列。代数数据类型里面也有相似的东西，一般称之为枚举类型。

以下是一个 enum 例子：
```
enum roygbiv {
    red,
    orange,
    yellow,
    green,
    blue,
    indigo,
    violet,
};
```
以下是等价的 Haskell 代码：

```
data Roygbiv = Red
             | Orange
             | Yellow
             | Green
             | Blue
             | Indigo
             | Violet
```
enum 的问题是，它使用整数值去代表元素：在一些接受 enum 的场景里，可以将整数传进去，C 编译器会自动进行类型转换。同样，在使用整数的场景里，也可以将一个 enum 元素传进去。这种用法可能会造成一些令人不爽的 bug 。

另一方面，在 Haskell 里就没有这样的问题。比如说，不可能使用 Roygbiv 里的某个值来代替 Int 值[译注：因为枚举类型的每个元素都由一个唯一的值构造器生成，而不是使用整数表示。]：

### 联合
如果一个代数数据类型有多个备选，那么可以将它看作是 C 或 C++ 里的 union 。

以上两者的一个主要区别是， union 并不告诉用户，当前使用的是哪一个备选， union 的使用者必须自己记录这方面的信息（通常使用一个额外的域来保存），这意味着，如果搞错了备选的信息，那么对 union 的使用就会出错。

以下是一个 union 例子：
```
enum shape_type {
    shape_circle,
    shape_poly,
};

struct circle {
    struct vector centre;
    float radius;
};

struct poly {
    size_t num_vertices;
    struct vector *vertices;
};

struct shape
{
    enum shape_type type;
    union {
    struct circle circle;
    struct poly poly;
    } shape;
};
```
在上面的代码里， shape 域的值可以是一个 circle 结构，也可以是一个 poly 结构。 shape_type 用于记录目前 shape 正在使用的结构类型。

另一方面，Haskell 版本不仅简单，而且更为安全：

```
type Vector = (Double, Double)

data Shape = Circle Vector Double
           | Poly [Vector]
```

注意，我们不必像 C 语言那样，使用 shape_type 域来手动记录 Shape 类型的值是由 Circle 构造器生成的，还是由 Poly 构造器生成， Haskell 自己有办法弄清楚一点，它不会弄混两种不同的值。其中的原因，下一节《模式匹配》就会讲到。

[译注：原文这里将 Poly 写成了 Square 。]
- 个人认为，要从发展的角度来看，C/C++本就是从底层一路抽象上来，C里面的struct是要直接关联内存分配大小的，这意味着C注定无法在语义的表达上与Haskell角力，但是Haskell最终也是要编译到二进制代码在冯诺依曼的体系上执行的，只是编译器帮我们屏蔽了底层的内存分配要考虑的问题。所以，我当然想用Haskell去描述我的逻辑，但也不意味着无脑鼓吹，捧一贬一。

# 模式匹配
- 上面介绍了定义代数数据类型的方法。接下来说明怎样去处理这些类型的值
- 对于某个类型的值来说，应该可以做到以下两点:
    1. 如果这个类型有一个以上的值构造器，那么应该可以知道，这个值是由哪个构造器创建的。
    2. 如果一个值构造器包含不同的成分，那么应该有办法提取这些成分。
- 对于以上两个问题，Haskell有一个简单且有效的解决方式，那就是类型匹配。
- 模式匹配允许我们查看值的内部，并将值所包含的数据绑定到变量上。以下是一个对 Bool 类型值进行模式匹配的例子，它的作用和 not 函数一样：



In [6]:
myNot True = False
myNot False = True

- 对于名字相同的函数,Haskell允许定义函数在不同输入参数模式下的不同行为,当调用函数式，Haskell会去匹配同名不同模式的函数，如果匹配成功则调用执行。
- 对于每行等式，模式定义放在函数名之后， = 符号之前。
- 一个更复杂的例子，以下函数是计算出列表所有元素之和:

In [7]:
sumList (x:xs) = x+sumList xs
sumList [] = 0

Line 1: Use foldr
Found:
sumList (x : xs) = x + sumList xs
sumList [] = 0
Why not:
sumList xs = foldr (+) 0 xs

- 需要说明的是，在Haskell里，列表[1,2],其实只是(1:(2:[]))的一种简单的表示方式，其中`:`用于构造列表:
```
Prelude> []
[]

Prelude> 1:[]
[1]

Prelude> 1:2:[]
[1,2]
```
- 因此，当需要对一个列表进行匹配时，也可以使用`:`操作符，只不过这次不是用来构造列表，而是用来分解列表。
- 继续考虑sumList,如果用sumLis去求值[1,2]，首先[1,2]会匹配到(x:xs)，于是x被绑定1,xs被绑定[2],此时表达式就变成了1+(sumList([2]))，后续的情况就类似。

In [ ]:
sumList [1,2]
sum [1,2]

## 组成和解构
- 让我们梳理一下构造一个值和对值进行模式匹配之间的关系
    - 描述一个构造值的过程: 表达式 `Book 9 "Close Calls" ["John Long"]` 应用 `Book Int String [String]` 构造器到值 9 、 "Close Calls" 和 ["John Long"] 上面，从而产生一个新的 BookInfo 类型的值。
    - 而当我们对Book构造器进行模式匹配时，我们在逆转(reverse)它的构造过程: 首先，检查这个值是否由 Book 构造器生成 —— 如果是的话，那么就对这个值进行探查（inspect），并取出创建这个值时，提供给构造器的各个值。
    
- 考虑一下表达式 Book 9 "Close Calls" ["John Long"] 对模式 (Book id name authors) 的匹配是如何进行的：

    - 因为值的构造器和模式里的构造器相同，因此匹配成功。
    - 变量 id 被绑定为 9 。
    - 变量 name 被绑定为 Close Calls 。
    - 变量 authors 被绑定为 ["John Long"] 。
- 因为模式匹配的过程像是逆转一个值的构造(construction)过程，因此它有时候也被称为结构(deconstruction)

## 更进一步
- 对元组进行模式匹配的语法，和构造元组的语法很相似。
- 以下是一个可以返回三元组中最后一个元素的函数:

In [ ]:
third (a,b,c) = c
third (1,2,3)

- 模式匹配的“深度”并没有限制，以下模式会同时对元组和元组里的列表进行匹配,匹配失败则会报错

In [ ]:
complicated (True, a, x:xs,5) = (a,xs)
complicated (True, 1, [1,2,3], 5)
complicated (False, 1, [1,2,3], 5)

- 对代数数据类型的匹配，可以通过这个类型的值构造器来进行。用前面定义的BookInfo类型为例，对它可以使用它的Book构造器来进行:

In [ ]:
bookID      (Book id title authors) = id
bookTitle   (Book id title authors) = title
bookAuthors (Book id title authors) = authors


book = Book 3 "Probability Theory" ["E.T.H Jaynes"]
bookID book
bookTitle book
bookAuthors book

-- 字面值的对比规则对于列表和值构造器的匹配也适用，比如这里只匹配id=3的
bookIdEq3    (Book 3 title authors) = title
bookIdEq3 book    

## 模式匹配中的变量名命名
当你阅读那些进行模式匹配的函数时，经常会发现像是 (x:xs) 或是 (d:ds) 这种类型的名字。这是一个流行的命名规则，其中的 s 表示“元素的复数”。以 (x:xs) 来说，它用 x 来表示列表的第一个元素，剩余的列表元素则用 xs 表示。



## 通配符模式匹配
- 如果在匹配模式中我们不在乎某个值的类型，那么可以用下划线字符 “_” 作为符号来进行标识，它也叫做通配符。它的用法如下：

In [ ]:
nicerID      (Book id _     _      ) = id
nicerTitle   (Book _  title _      ) = title
nicerAuthors (Book _  _     authors) = authors

- 显然我们使用变量也是可以达到同样的匹配效果，但是使用通配符去匹配我们不会使用的值会使得可读性更高，所以当我们匹配列表中定义了变量但是函数体中却没有使用，Haskell编译器会给出一个警告，这往往意味着潜在的bug

## 穷举匹配模式和通配符
- 在给一个类型写一组匹配模式时，很重要的一点就是一定要涵盖构造器的所有可能情况。例如，如果我们需要探查一个列表，就应该写一个匹配非空构造器 (:) 的方程和一个匹配空构造器 [] 的方程。

- 假如我们没有涵盖所有情况会发生什么呢。下面，我们故意漏写对 [] 构造器的检查。

In [ ]:
badExample (x:xs) = x + badExample xs
badExample []

- 如果我们将其作用于一个不能匹配的值，运行时就会报错:

- 但如果某些情况，我们并不在乎某些特定的构造器，我们就可以用通配符匹配模式来定义一个默认行为:

In [ ]:
goodExample (x:xs) = x+goodExample xs
goodExample _ = 0

goodExample []
goodExample [1,2]

# 记录语法
- 给一个数据类型的每个成分写访问器函数是令人感觉重复且反味的事情:

In [ ]:
nicerID      (Book id _     _      ) = id
nicerTitle   (Book _  title _      ) = title
nicerAuthors (Book _  _     authors) = authors

- 我们把这种代码叫做“样板代码（boilerplate code）”：尽管是必需的，但是又长又烦。Haskell 程序员不喜欢样板代码。幸运的是，语言的设计者提供了避免这个问题的方法：我们在定义一种数据类型的同时，就可以定义好每个成分的访问器。（逗号的位置是一个风格问题，如果你喜欢的话，也可以把它放在每行的最后。）：

```
data Customer = Customer {
      customerID      :: CustomerID
    , customerName    :: String
    , customerAddress :: Address
    } deriving (Show)
```
    
-- 上面👆的写法和下面👇的写法是几乎完全一致的(所以我很好奇作者为什么会说几乎完全一致？什么情况下不一致？)

```
data Customer = Customer Int String [String]
                deriving (Show)

customerID :: Customer -> Int
customerID (Customer id _ _) = id

customerName :: Customer -> String
customerName (Customer _ name _) = name

customerAddress :: Customer -> [String]
customerAddress (Customer _ _ address) = address
```

- 记录语法还新增了一种更详细的标识法来新建一个值。这种标识法通常都会提升代码的可读性:

In [ ]:
customer1 = Customer 271828 "J.R. Hacker"
            ["255 Syntax Ct",
             "Milpitas, CA 95134",
             "USA"]
customer2 = Customer {
              customerID = 271828
            , customerAddress = ["1048576 Disk Drive",
                                 "Milpitas, CA 95134",
                                 "USA"]
            , customerName = "Jane Q. Citizen"
            }

- 如果使用这种形式，我们还可以调换字段列表的顺序。比如在上面的例子里，name 和 address 字段的顺序就被移动过，和定义类型时的顺序不一样了。

- 当我们使用记录语法来定义类型时，还会影响到该类型的打印格式:

In [ ]:
customer1
customer2

-- 我们没有使用记录型语法时的打印格式:
cities = Book 173 "Use of Weapons" ["Iain M. Banks"]
cities

#  参数化类型
- 列表的类型是多态的，即列表中的元素可以是一致的任何类型。
- 我们也可以给自定义类型添加多态性:只要在类型定义中使用类型变量就可以做到这一点
- 常用的Maybe类型就使用了类型变量:
    ```
        data Maybe a = Just a
             | Nothing
    ```
- 这里的变量 a 不是普通的变量：它是一个类型变量。它意味着 `Maybe` 类型使用另一种类型作为它的参数。从而使得 Maybe 可以作用于任何类型的值:

In [ ]:
someBool = Just True
someString = Just "something"
someString
Just 1.5
Nothing

- Maybe 是一个多态，或者称作泛型的类型。我们向 Maybe 的类型构造器传入某种类型作为参数，例如 `Maybe Int` 或 `Maybe [Bool]`。 如我们所希望的那样，这些都是不同的类型（译注：可能省略了“但是都可以成功传入作为参数”）。

- 我们可以嵌套使用参数化的类型，但要记得使用括号标识嵌套的顺序，以便 Haskell 编译器知道如何解析这样的表达式。

# 递归类型

## 列表
- 列表这种常见的类型就是递归的：即它用自己来定义自己。为了深入了解其中的含义，让我们自己来设计一个与列表相仿的类型。我们将用 Cons 替换 (:) 构造器，用 Nil 替换 [] 构造器：

In [ ]:
data List a = Cons a (List a)
            | Nil
            deriving (Show)

- `List a`在=符号的左右两侧都有出现，我们可以说该类型的定义引用了它自己。当我们使用Cons构造器的时候，我们必须提供一个a的值作为参数一，以及一个`List a`类型的值作为参数二
- 由于 Nil 是一个 List a 类型（译注：原文是 List 类型，可能是漏写了 a），因此我们可以将它作为 Cons 的第二个参数。

In [ ]:
Nil
Cons 0 Nil
-- 这里it是ghci上一次执行结果
Cons 1 it
Cons 2 it
Cons 3 it

### Tip
- List可以被当作haskel里的list吗？
- 简单地证明一下List a类型和内置的list类型[a]拥有相同的构型。让我们设计一个函数能够接受任何一个[a]类型的值作为输入参数，并返回List a类型的一个值:

In [ ]:
fromList (x:xs) = Cons x (fromList xs)
fromList [] = Nil
fromList "durian"
fromList [Just True, Nothing, Just False]

- 上述代码可以将每个(:)替换成Cons(这也说明(:)实际上就是一种函数的语法糖),将每个[]替换成Nil。这样就涵盖了内置list类型的全部构造器。因此我们可以说二者是同构的，它们有着相同的构型

## 二叉树
- 为了更清楚说明什么是递归类型，继续说另一个例子--定义一个二叉树类型

In [ ]:
data Tree a = Node a (Tree a) (Tree a) 
            | Empty
            deriving (Show)

- 而如果要在Java中实现类似的定义:
```
class Tree<A> {
    A value;
    Tree<A> left;
    Tree<A> right;
    
    public Tree(A v, Tree<A> l, Tree<A> r) {
        this.value = v;
        this.left = l;
        this.right = r;
    }
}
``` 
- 上面是定义的不同，下面对比构造的不同:
    - java:
    ```
      new Tree<String>(
            "parent",
        new Tree<String>("left leaf", null, null),
        new Tree<String>("right leaf", null, null));
    }
    ```
    - Haskell:

In [ ]:
simpleTree = Node "parent" 
                (Node "left leaf" Empty Empty)
                (Node "right leaf" Empty Empty)

### 练习
1. 请给 List 类型写一个与 fromList 作用相反的函数：传入一个 List a 类型的值，返回一个 [a]:

In [ ]:
toList Cons a (List a) = a:(toList List a)
toList Nil = []
let myList = fromList [1,2,3]
let list = toList myList
myList
list